In [1]:
# Add % to work within conda environment
%pip install pulp
%conda install --yes numpy pandas seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

# All requested packages already installed.




==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0




Youngsu: After installing the packages, be sure to restart your kernel by pressing the circular icon above.
---

In [2]:
from pulp import *
import numpy as np
import pandas as pd 

In [3]:
n_demandnode = 7 # plus depot = 6  Max visit numbers for each location:
family0 = 1
family1 = np.array([2,3,4])
family2 = np.array([5,6,7])

#n_chargingstation = 1
n_drones = 2
n_slot = 7 ############ 7 before
monitor_time_matrix = np.array([3, 2, 2, 2, 1, 1, 1]) #Pj #[2, 1, 2, 3, 4, 5, 5]
Traveltime_demandnodes_matrix = np.array([[0,3,3,3,4,4,4 ],[3,0,0,0,2,2,2],[3,0,0,0,2,2,2],[3,0,0,0,2,2,2],[4,2,2,2,0,0,0],[4,2,2,2,0,0,0],[4,2,2,2,0,0,0]]) #Sjk
#max_intervisit_time_matrix = np.array([24, 5, 5, 5, 5]) #depot due date should be a large number.
B = 10000
#UB = 24 # upper bound of dj for Z
#LB = 0 # lower bound of dj for Z
due_dates = np.array([0, 5, 10, 15, 5, 10, 15]) #dj
Drone_Charge = 9 #8 before
UB = 1000 # upper bound of C 
intervisit_time = 10 # 5 before MAX
# Try to keep the scale of Traveltime_demandnodes_matrix and intervisit_time compatable! 
# if drone visits a location at time 1 it should visit the same location by the time 6 at least once. check! YES

In [4]:
for i in family1:
    print(i)

2
3
4


In [5]:
model = LpProblem("Parallel machines",LpMinimize)

C:\Users\otoragay\AppData\Local\miniconda3\envs\AMenv\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
Xjri_names = [str(j)+str(r)+str(i) for j in range(1, n_demandnode+1) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
#if customer j is assigned to the r slot of the drone i
#print("Xjri Indices:", Xjri_names)

In [7]:
Yjkri_names = [str(j)+str(k)+str(r)+str(i) for j in range(1, n_demandnode+1) for k in range (1, n_demandnode+1) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
# if customer j is assigned to the r slot and customer k is assigned to r-1 slot of drone i
#print("Yjkri Indices:", Yjkri_names)

In [8]:
Cri_names = [str(r)+str(i) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
#The Completion time of the monitoring customer on the r slot of the drone i
#print("Cri Indices:", Cri_names)

In [9]:
Zri_names = [str(r)+str(i) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
#The Completion time of the monitoring customer on the r slot of the drone i
#print("Zri Indices:", Zri_names)

In [10]:
Wri_names = [str(r)+str(i) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
#The Completion time of the monitoring customer on the r slot of the drone i
#print("Zri Indices:", Zri_names)

In [11]:
Vjri_names = [str(j)+str(r)+str(i) for j in range(1, n_demandnode+1) for r in range (1, n_slot+1) for i in range (1, n_drones+1)]
# Replacement for CriXjri
#print("Vjri Indices:", Vjri_names)

In [12]:
X = LpVariable.matrix("X", Xjri_names, cat = "Binary")
allocation1 = np.array(X).reshape(n_demandnode,n_slot,n_drones)
#print("Customer allocation j to slot r of drone i: ")
#print(allocation1)

In [13]:
Y = LpVariable.matrix("Y", Yjkri_names, cat = "Binary")
allocation2 = np.array(Y).reshape(n_demandnode,n_demandnode,n_slot,n_drones)
#print("Consequtive assignment of customer j on slot r after customer k, on drone i: ")
#print(allocation2)

In [14]:
C = LpVariable.matrix("C", Cri_names, cat="Integer")
allocation3 = np.array(C).reshape(n_slot,n_drones)
#print("completion time of monitoring the customer on slot r of drone i: ")
#print(allocation3)

In [15]:
Z = LpVariable.matrix("Z", Zri_names, cat="Binary")
allocation4 = np.array(Z).reshape(n_slot,n_drones)
#print("Consequtive assignment of charging on slot r of drone i: ")
#print(allocation4)

In [16]:
W = LpVariable.matrix("W", Wri_names, cat="Integer")
allocation5 = np.array(W).reshape(n_slot,n_drones)
#print("dummy variable for customer seved on slot r of drone i: ")
#print(allocation5)

In [17]:
V = LpVariable.matrix("V", Vjri_names, cat="Integer")
allocation6 = np.array(V).reshape(n_demandnode,n_slot,n_drones)
#print("dummy variable for customer j seved on slot r of drone i: ")
#print(allocation6)

In [18]:
Lmax= LpVariable ("Lmax", cat="Integer", lowBound= 0)  # when calculating Lmax do not include depot!!!
#print(Lmax)

In [19]:
#Completion = LpVariable ("Completion", cat="Continuous")  # when calculating Lmax do not include depot!!!
#print(Completion)

In [20]:
obj_func = Lmax #Completion
print(obj_func)
model +=  obj_func
print(model)

Lmax
Parallel_machines:
MINIMIZE
1*Lmax + 0
VARIABLES
0 <= Lmax Integer


In [21]:
#Constraint objective function
#print(Completion == lpSum(allocation3[r][i] for r in range (n_slot) for i in range (n_drones)))
#model += Completion == lpSum(allocation3[r][i] for r in range (n_slot) for i in range (n_drones))

In [22]:
#constratint 2 : each job should happen only once 
for j in range(1, n_demandnode):
   # print (lpSum(allocation1[j][r][i] for r in range (0, n_slot) for i in range (0, n_drones)) == 1)                
    model += lpSum(allocation1[j][r][i] for r in range (0, n_slot) for i in range (0, n_drones)) == 1

In [23]:
#constratint 3: 
#each slot on each drone can be empty or filled with a job or going to depot
# Later: Check being empty in the solution!!!!
for r in range(n_slot):
    for i in range(n_drones):
       # print (lpSum(allocation1[j][r][i] for j in range(1,n_demandnode)) <= 1)
        model += lpSum(allocation1[j][r][i] for j in range(1,n_demandnode)) <= 1

In [24]:
#constratint 4: only for depot X111?
#each slot on each drone can be empty or filled with a job or going to depot
# Later: Check being empty in the solution!!!!
for r in range(1,n_slot):
    for i in range(n_drones):
       # print (allocation1[0][r][i] <= 1)
        model += allocation1[0][r][i] <= 1

In [25]:
#constratint 5: Depot cannot happen in the first slot of each drone
for i in range(n_drones):
   # print (allocation1[0][0][i] == 0)
    model += allocation1[0][0][i] == 0

In [26]:
#constratint 6: no empty slots between jobs
for r in range(1,n_slot):
    for i in range(n_drones):
      #  print (lpSum(allocation1[j][r][i] - allocation1[j][r-1][i] for j in range(n_demandnode)) <= 0)
        model += lpSum(allocation1[j][r][i] - allocation1[j][r-1][i] for j in range(n_demandnode)) <= 0

In [27]:
#constratint 7 : removes repeated solutions: drones are identical
for i in range(n_drones-1):
   # print (lpSum(allocation1[j][r][i+1] - allocation1[j][r][i] for r in range (n_slot) for j in range(n_demandnode)) <= 0)
    model += lpSum(allocation1[j][r][i+1] - allocation1[j][r][i] for r in range (n_slot) for j in range(n_demandnode)) <= 0

In [28]:
#Traveltime_demandnodes_matrix

In [29]:
# Constraint to avoid depot at the first slot of each drone;
#print (allocation1[0][0][0] == 0)
#model += allocation1[0][0][0] == 0

In [30]:
# Constraint to avoid depot at the first slot of each drone;
#print (allocation1[0][0][1] == 0)
#model += allocation1[0][0][1] == 0

In [31]:
#constratint 8 This will take care of not having depot assigned to the first spot on each drone.
for i in range(n_drones):
  #  print (allocation3[0][i] == lpSum((Traveltime_demandnodes_matrix[0][j] + monitor_time_matrix[j])*allocation1[j][0][i] for j in range(1,n_demandnode)))
    model += allocation3[0][i] == lpSum((Traveltime_demandnodes_matrix[0][j]+monitor_time_matrix[j])*allocation1[j][0][i] for j in range(1,n_demandnode))

In [32]:
#constratint 9: Completion time
for i in range(n_drones):
    for r in range(1,n_slot):
       # print (allocation3[r][i]== (allocation3[r-1][i])+ lpSum( Traveltime_demandnodes_matrix[k][j]*allocation2[j][k][r][i] for j in range(n_demandnode) for k in range(n_demandnode) if j is not k)+lpSum(monitor_time_matrix[j]*allocation1[j][r][i] for j in range(n_demandnode)))
        model += allocation3[r][i]== (allocation3[r-1][i])+ lpSum( Traveltime_demandnodes_matrix[k][j]*allocation2[j][k][r][i] for j in range(n_demandnode) for k in range(n_demandnode) if j is not k)+lpSum(monitor_time_matrix[j]*allocation1[j][r][i] for j in range(n_demandnode))
        

In [33]:
#constratint 10: Defines Y
for i in range(n_drones):
    for j in range(n_demandnode):
        for k in range(n_demandnode):
            if j is not k:
                for r in range(1,n_slot):
                    #print (lpSum(allocation1[j][r][i]+allocation1[k][r-1][i]-1)<=allocation2[j][k][r][i])
                    model += lpSum(allocation1[j][r][i]+allocation1[k][r-1][i]-1)<=allocation2[j][k][r][i]

In [34]:
#constratint 11: defines Y
for i in range(n_drones):
    for j in range(n_demandnode):
        for k in range(n_demandnode):
            if j is not k:
                for r in range(1,n_slot):
                    #print (lpSum(0.5*(allocation1[j][r][i]+allocation1[k][r-1][i])- allocation2[j][k][r][i])>=0)
                    model += lpSum(0.5*(allocation1[j][r][i]+allocation1[k][r-1][i])- allocation2[j][k][r][i])>=0

In [35]:
#constratint 12:  Lmax original  -B*(1-lpSum(allocation1[j][r][i] for j in range(n_demandnode)))

for r in range(n_slot):
    for i in range(n_drones):
       # print(Lmax>=allocation3[r][i]-lpSum(due_dates[j]*allocation1[j][r][i] for j in range(n_demandnode))-B*(1-lpSum(allocation1[j][r][i] for j in range(n_demandnode))))
        model += Lmax>=allocation3[r][i]-lpSum(due_dates[j]*allocation1[j][r][i] for j in range(n_demandnode))-B*(1-lpSum(allocation1[j][r][i] for j in range(n_demandnode)))

In [36]:
# New constarint for C
#for r in range(n_slot):
  #  for i in range(n_drones):
   #     print(allocation3[r][i] <= B*lpSum(allocation1[j][r][i] for j in range(n_demandnode)))
   #     model += allocation3[r][i] <= B*lpSum(allocation1[j][r][i] for j in range(n_demandnode))

In [37]:
# Constraint: drone should go to charging station if the completion time exceeds charge available on drone
# Try to work with Drone_Charge    *lpSum(1+allocation4[r][i] for r in range(0,a)) 
#### Didn't read the last slot!!!
#### shouldn't have sum over c: C31 already has c11 + c21 in it!!
#for i in range(n_drones):
 #   for r in range(n_slot-1):
    #    print(allocation3[r][i] - Drone_Charge*(1+lpSum(allocation4[a][i] for a in range(0,r+1))) <= B*allocation4[r+1][i])
    #    model += allocation3[r][i] - Drone_Charge*(1+lpSum(allocation4[a][i] for a in range(0,r+1))) <= B*allocation4[r+1][i]

In [38]:
# Constraint 13

# TEST Constraint: drone should go to charging station if the completion time exceeds charge available on drone
# Try to work with Drone_Charge    *lpSum(1+allocation4[r][i] for r in range(0,a)) 
#### Didn't read the last slot!!!
#### shouldn't have sum over c: C31 already has c11 + c21 in it!!
for i in range(n_drones):
        print(allocation3[0][i] - Drone_Charge <= B*allocation4[1][i])
        model += allocation3[0][i] - Drone_Charge <= B*allocation4[1][i]

C_11 - 10000*Z_21 <= 9
C_12 - 10000*Z_22 <= 9


In [39]:
# Constraint 14 below!!!

In [40]:
# Constraint 15

# TESt 2Constraint: drone should go to charging station if the completion time exceeds charge available on drone
# Try to work with Drone_Charge    *lpSum(1+allocation4[r][i] for r in range(0,a)) 
#### Didn't read the last slot!!!
#### shouldn't have sum over c: C31 already has c11 + c21 in it!!
for i in range(n_drones):
    for r in range(1,n_slot-1):
        print(allocation3[r][i] - lpSum(allocation5[b][i] for b in range(0,r)) - Drone_Charge <= B*allocation4[r+1][i])
        model += allocation3[r][i] - lpSum(allocation5[b][i] for b in range(0,r)) - Drone_Charge <= B*allocation4[r+1][i]

C_21 - W_11 - 10000*Z_31 <= 9
C_31 - W_11 - W_21 - 10000*Z_41 <= 9
C_41 - W_11 - W_21 - W_31 - 10000*Z_51 <= 9
C_51 - W_11 - W_21 - W_31 - W_41 - 10000*Z_61 <= 9
C_61 - W_11 - W_21 - W_31 - W_41 - W_51 - 10000*Z_71 <= 9
C_22 - W_12 - 10000*Z_32 <= 9
C_32 - W_12 - W_22 - 10000*Z_42 <= 9
C_42 - W_12 - W_22 - W_32 - 10000*Z_52 <= 9
C_52 - W_12 - W_22 - W_32 - W_42 - 10000*Z_62 <= 9
C_62 - W_12 - W_22 - W_32 - W_42 - W_52 - 10000*Z_72 <= 9


In [41]:
# Constraint 16 below!!! 

In [42]:
# Define W: 17
for i in range(n_drones):
    for r in range(2,n_slot):
        print(allocation5[r-1][i] >= allocation3[r-1][i] - UB*(1-allocation4[r][i]))
        model += allocation5[r-1][i] >= allocation3[r-1][i] - UB*(1-allocation4[r][i])

-C_21 + W_21 - 1000*Z_31 >= -1000
-C_31 + W_31 - 1000*Z_41 >= -1000
-C_41 + W_41 - 1000*Z_51 >= -1000
-C_51 + W_51 - 1000*Z_61 >= -1000
-C_61 + W_61 - 1000*Z_71 >= -1000
-C_22 + W_22 - 1000*Z_32 >= -1000
-C_32 + W_32 - 1000*Z_42 >= -1000
-C_42 + W_42 - 1000*Z_52 >= -1000
-C_52 + W_52 - 1000*Z_62 >= -1000
-C_62 + W_62 - 1000*Z_72 >= -1000


In [43]:
# Define W: 18  ## we do not consider r= 7 the last slot
for i in range(n_drones):
    for r in range(2,n_slot):
        print(allocation5[r-1][i] <= allocation3[r-1][i])
        model += allocation5[r-1][i] <= allocation3[r-1][i]
        

-C_21 + W_21 <= 0
-C_31 + W_31 <= 0
-C_41 + W_41 <= 0
-C_51 + W_51 <= 0
-C_61 + W_61 <= 0
-C_22 + W_22 <= 0
-C_32 + W_32 <= 0
-C_42 + W_42 <= 0
-C_52 + W_52 <= 0
-C_62 + W_62 <= 0


In [44]:
# Define W: 19
for i in range(n_drones):
    for r in range(2,n_slot):
        print(allocation5[r-1][i] <= UB*allocation4[r][i])
        model += allocation5[r-1][i] <= UB*allocation4[r][i]
        

W_21 - 1000*Z_31 <= 0
W_31 - 1000*Z_41 <= 0
W_41 - 1000*Z_51 <= 0
W_51 - 1000*Z_61 <= 0
W_61 - 1000*Z_71 <= 0
W_22 - 1000*Z_32 <= 0
W_32 - 1000*Z_42 <= 0
W_42 - 1000*Z_52 <= 0
W_52 - 1000*Z_62 <= 0
W_62 - 1000*Z_72 <= 0


In [45]:
# Define W: 20


#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ OCT 27
#for i in range(n_drones):
 #   for r in range(1,n_slot+1):
        #print(allocation5[r-1][i] >= 0)
  #      model += allocation5[r-1][i] >= 0
        

In [46]:
# Define W: 21 : means that Z_(2,i) cannot be 1!!! Check if this is OK!
# Shoul dwork even without this cause Z11 = Z21 = 0 always
for i in range(n_drones):
    #print(allocation5[0][i] == 0 )
    model += allocation5[0][i] == 0

In [47]:
# Constraint 14: drone should go to charging station if the completion time exceeds charge available on drone
# Needs two constraint to inforce Z=0 if Drone charge is still available: *lpSum(1+allocation4[r][i] for r in range(0,a))
# Try to work with Drone_Charge
#### Didn't read the last slot!!! @@@@@@@

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ OCT 20 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Removed
for i in range(n_drones):
    print(allocation3[0][i] - Drone_Charge  >= -B*(1-allocation4[1][i]))
    model += allocation3[0][i] - Drone_Charge  >= -B*(1-allocation4[1][i])

C_11 - 10000*Z_21 >= -9991
C_12 - 10000*Z_22 >= -9991


In [48]:
# Constraint 16: drone should go to charging station if the completion time exceeds charge available on drone
# Needs two constraint to inforce Z=0 if Drone charge is still available: *lpSum(1+allocation4[r][i] for r in range(0,a))
# Try to work with Drone_Charge
#### Didn't read the last slot!!!

#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ OCT 20 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Removed

for i in range(n_drones):
    for r in range(1,n_slot-1):
        print(allocation3[r][i] - lpSum(allocation5[b][i] for b in range(0,r)) - Drone_Charge  >= -B*(1-allocation4[r+1][i]))
        model += allocation3[r][i] - lpSum(allocation5[b][i] for b in range(0,r)) - Drone_Charge  >= -B*(1-allocation4[r+1][i])

C_21 - W_11 - 10000*Z_31 >= -9991
C_31 - W_11 - W_21 - 10000*Z_41 >= -9991
C_41 - W_11 - W_21 - W_31 - 10000*Z_51 >= -9991
C_51 - W_11 - W_21 - W_31 - W_41 - 10000*Z_61 >= -9991
C_61 - W_11 - W_21 - W_31 - W_41 - W_51 - 10000*Z_71 >= -9991
C_22 - W_12 - 10000*Z_32 >= -9991
C_32 - W_12 - W_22 - 10000*Z_42 >= -9991
C_42 - W_12 - W_22 - W_32 - 10000*Z_52 >= -9991
C_52 - W_12 - W_22 - W_32 - W_42 - 10000*Z_62 >= -9991
C_62 - W_12 - W_22 - W_32 - W_42 - W_52 - 10000*Z_72 >= -9991


In [49]:
# Constraint 22 connecting Z to X: X is in objective function but does it prevent Z to be 1 all the time?
for i in range(n_drones):
    for r in range(n_slot):
        #print(allocation1[0][r][i] == allocation4[r][i])
        model += allocation1[0][r][i] == allocation4[r][i]

In [50]:
# Costriant 22 - 1 Continue from HEREEEEE
#Job family 1 constraint: jobs 1,2,3,4  ---- j {2,3,4,5,6,7}

for j in range(1,3):
#for j in family1:
    print(lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) <= intervisit_time)
    model += lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) <= intervisit_time


-V_211 - V_212 - V_221 - V_222 - V_231 - V_232 - V_241 - V_242 - V_251 - V_252 - V_261 - V_262 - V_271 - V_272 + V_311 + V_312 + V_321 + V_322 + V_331 + V_332 + V_341 + V_342 + V_351 + V_352 + V_361 + V_362 + V_371 + V_372 <= 10
-V_311 - V_312 - V_321 - V_322 - V_331 - V_332 - V_341 - V_342 - V_351 - V_352 - V_361 - V_362 - V_371 - V_372 + V_411 + V_412 + V_421 + V_422 + V_431 + V_432 + V_441 + V_442 + V_451 + V_452 + V_461 + V_462 + V_471 + V_472 <= 10


In [51]:
# Costriant 23 -1

#Job family 1 constraint: jobs 1,2,3,4 non negative
for j in range(1,3):
    #print(lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) >= 1)
    model += lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) >= 1


In [52]:
# Costriant 22 - 2

#Job family 2 constraint: jobs 5,6,7
for j in range(4,6):
    #print(lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) <= intervisit_time)
    model += lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) <= intervisit_time


In [53]:
# Costriant 23 -2

#Job family 2 constraint: jobs 5,6,7 non negative
for j in range(4,6):
    #print(lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) >= 1)
    model += lpSum(allocation6[j+1][r][i] - allocation6[j][r][i] for r in range(n_slot) for i in range(n_drones)) >= 1


In [54]:
#Job family constraint: #Dummy 1
for j in range(n_demandnode):
    for r in range(n_slot):
        for i in range(n_drones):
            #print(allocation6[j][r][i] <= UB * allocation1[j][r][i])
            model += allocation6[j][r][i] <= UB * allocation1[j][r][i]

In [55]:
#Job family constraint: #Dummy 2
for j in range(n_demandnode):
    for r in range(n_slot):
        for i in range(n_drones):
            #print(allocation6[j][r][i] <= allocation3[r][i])
            model += allocation6[j][r][i] <= allocation3[r][i]

In [56]:
#Job family constraint: #Dummy 3
for j in range(n_demandnode):
    for r in range(n_slot):
        for i in range(n_drones):
            #print(allocation6[j][r][i] >= allocation3[r][i] - UB *(1-allocation1[j][r][i]))
            model += allocation6[j][r][i] >= allocation3[r][i] - UB *(1-allocation1[j][r][i])

In [57]:
#Job family constraint: #Dummy 4
for j in range(n_demandnode):
    for r in range(n_slot):
        for i in range(n_drones):
            #print(allocation6[j][r][i] >= 0)
            model += allocation6[j][r][i] >= 0

In [58]:
# Job family 1: Break sequence of the same family 2,4,3,...   @@@@@@@@@@@@@@@@@@@@@@@@@ Correct constraint structure
for j in range(1,3):
    for r in range(n_slot-1):
        for i in range(n_drones):
            print(allocation1[j+1][r+1][i] <= B*(1-allocation1[j][r][i] ))
            model += allocation1[j+1][r+1][i]  <= B*(1-allocation1[j][r][i])

10000*X_211 + X_321 <= 10000
10000*X_212 + X_322 <= 10000
10000*X_221 + X_331 <= 10000
10000*X_222 + X_332 <= 10000
10000*X_231 + X_341 <= 10000
10000*X_232 + X_342 <= 10000
10000*X_241 + X_351 <= 10000
10000*X_242 + X_352 <= 10000
10000*X_251 + X_361 <= 10000
10000*X_252 + X_362 <= 10000
10000*X_261 + X_371 <= 10000
10000*X_262 + X_372 <= 10000
10000*X_311 + X_421 <= 10000
10000*X_312 + X_422 <= 10000
10000*X_321 + X_431 <= 10000
10000*X_322 + X_432 <= 10000
10000*X_331 + X_441 <= 10000
10000*X_332 + X_442 <= 10000
10000*X_341 + X_451 <= 10000
10000*X_342 + X_452 <= 10000
10000*X_351 + X_461 <= 10000
10000*X_352 + X_462 <= 10000
10000*X_361 + X_471 <= 10000
10000*X_362 + X_472 <= 10000


In [59]:
# Job family 2: Break sequence of the same family 2,4,3,... @@@@@@@@@@@@@@@@@@@@@@@@@ Correct constraint structure
for j in range(4,6):
    for r in range(n_slot-1):
        for i in range(n_drones):
            print(allocation1[j+1][r+1][i]  <= B*(1-allocation1[j][r][i]))
            model += allocation1[j+1][r+1][i]  <= B*(1-allocation1[j][r][i] )

10000*X_511 + X_621 <= 10000
10000*X_512 + X_622 <= 10000
10000*X_521 + X_631 <= 10000
10000*X_522 + X_632 <= 10000
10000*X_531 + X_641 <= 10000
10000*X_532 + X_642 <= 10000
10000*X_541 + X_651 <= 10000
10000*X_542 + X_652 <= 10000
10000*X_551 + X_661 <= 10000
10000*X_552 + X_662 <= 10000
10000*X_561 + X_671 <= 10000
10000*X_562 + X_672 <= 10000
10000*X_611 + X_721 <= 10000
10000*X_612 + X_722 <= 10000
10000*X_621 + X_731 <= 10000
10000*X_622 + X_732 <= 10000
10000*X_631 + X_741 <= 10000
10000*X_632 + X_742 <= 10000
10000*X_641 + X_751 <= 10000
10000*X_642 + X_752 <= 10000
10000*X_651 + X_761 <= 10000
10000*X_652 + X_762 <= 10000
10000*X_661 + X_771 <= 10000
10000*X_662 + X_772 <= 10000


In [60]:
# Limit the operational time of each drone to 24 hours: @@@@@@@@@@@@@@@@@@@@@@@@@ We can use it for larger problems. 
#for i in range(n_drones):
 #   print(lpSum(allocation6[j][r][i] for j in range(n_demandnode) for r in range(n_slot)) <= 24)
  #  model += lpSum(allocation6[j][r][i] for j in range(n_demandnode) for r in range(n_slot)) <= 24

In [61]:
# Following two lines list available solvers 
# and we use the threads option to use multiple cores

print(listSolvers(onlyAvailable=True))

cbc_solver = PULP_CBC_CMD(threads=64) # change thereads=4 to the number you need 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-29
['GLPK_CMD', 'GUROBI_CMD', 'PULP_CBC_CMD']


C:\Users\otoragay\AppData\Local\miniconda3\envs\AMenv\lib\site-packages\pulp\apis\gurobi_api.py:170: UserWarning: GUROBI error: Unknown parameter '_test'.
  warnings.warn("GUROBI error: {}.".format(e))


In [ ]:
%%time

#model.solve()
#model.solve(PULP_CBC_CMD())
model.solve(cbc_solver)

status =  LpStatus[model.status]

print(status)

#solver.wall_time()


In [ ]:
#import pulp as pl
#solver_list = pl.listSolvers(onlyAvailable=True)
#solver = pl.getSolver('CPLEX_CMD')
#solver = pl.getSolver('GUROBI')